In [168]:
import numpy as np
import time
import timeit
import numba

Originale Version

In [169]:
def find_nearest_partners_original(values1, values2):
    """Detect the closest match in values2 for each number in values1."""
    nearest_partners = []
    for i, num1 in enumerate(values1):
        min_distance = np.inf  # start with infinity
        partner_id = 0
        # Compare to all numbers in values2 and find best match
        for j, num2 in enumerate(values2):
            # If better match than before:
            if abs(num1 - num2) < min_distance:
                min_distance = abs(num1 - num2)
                partner_id = j
        nearest_partners.append([i, partner_id, min_distance])
    return nearest_partners

Erstes Profiling (mit time)

In [170]:
if __name__ == "__main__":
    values1 = np.random.random(1000)
    values2 = np.random.random(1000)

    start = time.time()
    find_nearest_partners_original(values1, values2)
    print(time.time() - start)

0.3420233726501465


**Numpy Version**

Beschreibung: 

Diese Funktion macht aus den Daten eine Matrix. Die Matrix v1 wiederholt values1 in jeder Spalte. v2 wiederholt values 2 in jeder Zeile.

Wenn wir diese Matrizen subtrahieren, so erhalten wir eine Differenzmatrix, die Zeile gibt dabei den Index von values1 an, die Spalte von values2.

Indem wir dann das minimum jeder Zeile (entlang der Spalten) suchen, erhalten wir den Index des jeweiligen Nachbarn.

Am Ende geben wir alle Indices von values1, mitsammt ihres Nachbarn und dem Abstand zurück.

In [171]:
def find_nearest_partners_numpy(values1, values2):
    v1 = np.repeat(values1[:, None], values2.shape[0], axis=1) # values1 entlang der Zeilen kopieren
    v2 = np.repeat(values2[None, :], values1.shape[0], axis=0) # values2 entlang der Spalten kopieren

    diff = np.abs(v1-v2) #Zeilen:values1, spalten: values2

    nearest_neighbor = np.argmin(diff, axis=1) #zeigt für jeden value1 den nahesten value2

    return np.column_stack((np.arange(values1.shape[0]), nearest_neighbor, diff[np.arange(values1.shape[0]), nearest_neighbor]))

**Numba-Version**

Jit auf der ursprünglichen Version

Beschreibung:

WIr dekorieren die ursprüngliche Version mithilfe der numba Bibliothek.
Als dekorator benutzen wir @numba.jit

In [172]:
@numba.jit
def find_nearest_partners_numba_jit(values1, values2):
    """Detect the closest match in values2 for each number in values1."""
    nearest_partners = []
    for i, num1 in enumerate(values1):
        min_distance = np.inf  # start with infinity
        partner_id = 0
        # Compare to all numbers in values2 and find best match
        for j, num2 in enumerate(values2):
            # If better match than before:
            if abs(num1 - num2) < min_distance:
                min_distance = abs(num1 - num2)
                partner_id = j
        nearest_partners.append([i, partner_id, min_distance])
    return nearest_partners

Njit auf der ursprünglichen version

Beschreibung:

WIr dekorieren die ursprüngliche Version mithilfe der numba Bibliothek.
Als dekorator benutzen wir @numba.njit

In [173]:
@numba.jit
def find_nearest_partners_numba_njit(values1, values2):
    """Detect the closest match in values2 for each number in values1."""
    nearest_partners = []
    for i, num1 in enumerate(values1):
        min_distance = np.inf  # start with infinity
        partner_id = 0
        # Compare to all numbers in values2 and find best match
        for j, num2 in enumerate(values2):
            # If better match than before:
            if abs(num1 - num2) < min_distance:
                min_distance = abs(num1 - num2)
                partner_id = j
        nearest_partners.append([i, partner_id, min_distance])
    return nearest_partners

**Performance Vergleich**

In [174]:
if __name__ == "__main__":
    values1 = np.random.random(1000)
    values2 = np.random.random(1000)

# Zur kompilierung
find_nearest_partners_numba_jit(values1, values2)
find_nearest_partners_numba_njit(values1, values2)

n = 20

print( timeit.timeit(lambda:find_nearest_partners_original(values1, values2), number=n) / n)
print( timeit.timeit(lambda:find_nearest_partners_numpy(values1, values2), number=n) / n)
print( timeit.timeit(lambda:find_nearest_partners_numba_jit(values1, values2), number=n) / n)
print( timeit.timeit(lambda:find_nearest_partners_numba_njit(values1, values2), number=n) / n)

0.3114632850047201
0.019212270015850665
0.0012784349964931607
0.0012062449939548969


**Fazit**

Ein Vergleich der Laufzeiten ergibt:

Original: 0.3238932649837807
Numpy: 0.01666730500292033
Jit: 0.001553385006263852
Njit: 0.0013738750014454127

Wie zu erwarten war die langsamste version von allen die originale. Diese ist vorallem dadurch ineffizient, dass sir klassische Python Build-in funktionen benutzt und mit Schleifen arbeitet.

Die Numpy Version ist um einiges schneller, da Numpy effizienter arbeitet als klassisches Python(Wird teils durch C kompiliert). Außerdem sind Matrix operationen mathematisch sehr effizient. Diese hat aber zusätlichen Overhead, dadurch dass die Matrizen erstmals erstellt werden müssen.

Jit: Obwohl hier ebefnalls mit Schleifen gearbeitet wird, ist diese version um einiges schneller als numpy, da Numba den Code in maschinellen Code übersetzt. Wobei hier ein einmaliger Kompilierungsoverhead besteht.

Njit: Diese Version ist minimal schneller als die Jit version und überführt den Code ebenfalls in maschinellen Code. Auch hier besteht ein einmaloiger Kompilierungsoverhead.